In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch

from torch.utils.data import Dataset, DataLoader, random_split
from datasets import load_dataset
import pandas as pd
from scripts.clip_rn50 import ClipFinetuner, train, CustomDataset, evaluate
from scripts.datasets import office_home, convert_bytes_to_images
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import numpy as np

from vendi_score import image_utils
import weightwatcher as ww
import warnings

In [ ]:
ds = load_dataset("flwrlabs/office-home")

class_names, real_data = office_home('real')
print(f"Class names: {class_names}")

In [ ]:
real_data_df = real_data.to_pandas()

# Split the in-distribution data into training (80%), validation (10%), and testing (10%)
train_val_df, test_df = train_test_split(
    real_data_df, 
    test_size=0.2, 
    random_state=42, 
    stratify=real_data_df['label']
)
train_df, val_df = train_test_split(
    train_val_df, 
    test_size=0.125, 
    random_state=42, 
    stratify=train_val_df['label']
)

from datasets import Dataset
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)
test_data = Dataset.from_pandas(test_df)

print(f"In-Distribution Training Data: {len(train_data)}")
print(f"In-Distribution Validation Data: {len(val_data)}")
print(f"In-Distribution Test Data: {len(test_data)}")

In [4]:
train_images = train_data['image']
train_labels = train_data['label']
classes = ds['train'].features['label'].names
num_classes = len(set(classes))

val_images = val_data['image']
val_labels = val_data['label']

test_images = test_data['image']
test_labels = test_data['label']

train_images = convert_bytes_to_images(train_images)
val_images = convert_bytes_to_images(val_images)
test_images = convert_bytes_to_images(test_images)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for CLIP
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))  # CLIP's mean and std
])

In [5]:
acc = {}
loss = {}
ece = {}

# Create custom datasets
train_dataset = CustomDataset(images=train_images, labels=train_labels, classes=classes, transform=transform)
val_dataset = CustomDataset(images=val_images, labels=val_labels, classes=classes, transform=transform)
test_dataset = CustomDataset(images=test_images, labels=test_labels, classes=classes, transform=transform)

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_classes = len(set(train_labels))
model = ClipFinetuner(num_classes=num_classes)

torch.save(model.state_dict(), 'ft_rn50/model/model_bs.pth') #path

In [ ]:
watcher = ww.WeightWatcher(model=model)
details = watcher.analyze(mp_fit=True)
summary = watcher.get_summary(details)

df_bs = pd.DataFrame([summary])
df_bs.to_csv('ft_rn50/ww-sum/bs_summary.csv', index=False)

details.to_csv('ft_rn50/ww-df/bs.csv', index=False)

learning_rate = 1e-5
epochs = 5

ft_model = train(model, train_dataloader, val_dataloader, num_epochs=epochs)

test_loss, test_acc, test_ece = evaluate(ft_model, test_dataloader)
print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
print(f'Test ECE: {test_ece:.3f}')
acc['bs'] = test_acc
loss['bs'] = test_ece
ece['bs'] = test_ece

watcher_ft = ww.WeightWatcher(model=ft_model)
details_ft = watcher_ft.analyze(mp_fit=True)
summary_ft = watcher_ft.get_summary(details_ft)

torch.save(ft_model.state_dict(), 'ft_rn50/model/ft_model_bs.pth')

df_bs_ft = pd.DataFrame([summary_ft])
df_bs_ft.to_csv('ft_rn50/ww-sum/ft_summary_bs.csv', index=False)

details_ft .to_csv('ft_rn50/ww-df/ft_bs.csv', index=False)

DP

In [ ]:
dropout_rate = [0.1, 0.3, 0.5, 0.7]
# Dictionaries to store models, watchers, and summaries
model_dp = {}
watcher_dp = {}
details_dp = {}
summary_dp = {}
ft_model_dp = {}
watcher_ft_dp = {}
details_ft_dp = {}
summary_ft_dp = {}


# Dictionaries to store evaluation metrics
test_loss_dict_dp = {}
test_acc_dict_dp = {}
test_ece_dict_dp = {}

for i in dropout_rate:
    # Define models and watchers
    model_dp[i] = ClipFinetuner(num_classes=num_classes, block_dropout_rate=i, classifier_dropout_rate=i)
    watcher_dp[i] = ww.WeightWatcher(model=model_dp[i])

    # Perform analysis and get summary
    details_dp[i] = watcher_dp[i].analyze(mp_fit=True)
    summary_dp[i] = watcher_dp[i].get_summary(details_dp[i])

    # Save model state
    torch.save(model_dp[i].state_dict(), f'ft_rn50/model/model_dp{i}.pth')

    # Save analysis results to CSV
    details_dp[i].to_csv(f'ft_rn50/ww-df/dp{i}.csv', index=False)
  
    # Train the model
    ft_model_dp[i] = train(model_dp[i], train_dataloader, val_dataloader, num_epochs=epochs)

    # Evaluate the model
    test_loss, test_acc, test_ece = evaluate(ft_model_dp[i], test_dataloader)
    print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
    print(f'Test ECE: {test_ece:.3f}')

    # Store metrics
    test_loss_dict_dp[i] = test_loss
    test_acc_dict_dp[i] = test_acc
    test_ece_dict_dp[i] = test_ece

    # Analyze fine-tuned model
    watcher_ft_dp[i] = ww.WeightWatcher(model=ft_model_dp[i])
    details_ft_dp[i] = watcher_ft_dp[i].analyze(mp_fit=True)
    summary_ft_dp[i] = watcher_ft_dp[i].get_summary(details_ft_dp[i])

    # Save fine-tuned model state
    torch.save(ft_model_dp[i].state_dict(), f'ft_rn50/model/ft_model_dp{i}.pth')
  
    # Save fine-tuned analysis results to CSV
    details_ft_dp[i].to_csv(f'ft_rn50/ww-df/ft_dp{i}.csv', index=False)
  

acc.update(test_acc_dict_dp)
loss.update(test_loss_dict_dp)
ece.update(test_ece_dict_dp)

sum_dp = pd.DataFrame([summary_dp])
sum_dp_ft = pd.DataFrame([summary_ft_dp])

sum_dp.to_csv('ft_rn50/ww-sum/sum_dp.csv', index=False)

sum_dp_ft.to_csv('ft_rn50/ww-sum/ft_sum_dp.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
baseline_loss = test_loss
baseline_accuracy = test_acc
baseline_ece = test_ece

plt.subplot(1, 3, 1)
plt.plot(dropout_rate, [test_acc_dict_dp[i] for i in dropout_rate], marker='o', label='Accuracy')
plt.axhline(y=baseline_accuracy, color='black', linestyle='--', label='Baseline Accuracy')
plt.title('Accuracy vs Dropout Rate')
plt.xlabel('Dropout Rate')
plt.ylabel('Accuracy (%)')
plt.legend()

# Plot Loss
plt.subplot(1, 3, 2)
plt.plot(dropout_rate, [test_loss_dict_dp[i] for i in dropout_rate], marker='o', label='Loss')
plt.axhline(y=baseline_loss, color='black', linestyle='--', label='Baseline Loss')
plt.title('Loss vs Dropout Rate')
plt.xlabel('Dropout Rate')
plt.ylabel('Loss')
plt.legend()

# Plot ECE
plt.subplot(1, 3, 3)
plt.plot(dropout_rate, [test_ece_dict_dp[i] for i in dropout_rate], marker='o', label='ECE')
plt.axhline(y=baseline_ece, color='black', linestyle='--', label='Baseline ECE')
plt.title('ECE vs Dropout Rate')
plt.xlabel('Dropout Rate')
plt.ylabel('ECE')
plt.legend()

plt.tight_layout()
plt.show()

WD

In [ ]:
weight_decay_values = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3]

model_wd = {}
watcher_wd = {}
details_wd = {}
summary_wd = {}
ft_model_wd = {}
watcher_ft_wd = {}
details_ft_wd = {}
summary_ft_wd = {}


# Dictionaries to store evaluation metrics
test_loss_dict_wd = {}
test_acc_dict_wd = {}
test_ece_dict_wd = {}

for i in weight_decay_values:
    # Define models and watchers
    model_wd[i] = ClipFinetuner(num_classes=num_classes)
    watcher_wd[i] = ww.WeightWatcher(model=model_wd[i])

    # Perform analysis and get summary
    details_wd[i] = watcher_wd[i].analyze(mp_fit=True)
    summary_wd[i] = watcher_wd[i].get_summary(details_wd[i])

    # Save model state
    torch.save(model_wd[i].state_dict(), f'ft_rn50/model/model_wd{i}.pth')


    # Save analysis results to CSV
    details_wd[i].to_csv(f'ft_rn50/ww-df/wd{i}.csv', index=False)

    # Train the model
    ft_model_wd[i] = train(model_wd[i], train_dataloader, val_dataloader, num_epochs=epochs, weight_decay=i)

    # Evaluate the model
    test_loss, test_acc, test_ece = evaluate(ft_model_wd[i], test_dataloader)
    print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
    print(f'Test ECE: {test_ece:.3f}')

    # Store metrics
    test_loss_dict_wd[i] = test_loss
    test_acc_dict_wd[i] = test_acc
    test_ece_dict_wd[i] = test_ece

    # Analyze fine-tuned model
    watcher_ft_wd[i] = ww.WeightWatcher(model=ft_model_wd[i])
    details_ft_wd[i] = watcher_ft_wd[i].analyze(mp_fit=True)
    summary_ft_wd[i] = watcher_ft_wd[i].get_summary(details_ft_wd[i])

    # Save fine-tuned model state
    torch.save(ft_model_wd[i].state_dict(), f'ft_rn50/model/ft_model_wd{i}.pth')

    # Save fine-tuned analysis results to CSV
    details_ft_wd[i].to_csv(f'ft_rn50/ww-df/ft_wd{i}.csv', index=False)

acc.update(test_acc_dict_wd)
loss.update(test_loss_dict_wd)
ece.update(test_ece_dict_wd)

sum_wd = pd.DataFrame([summary_wd])
sum_wd_ft = pd.DataFrame([summary_ft_wd])

sum_wd.to_csv('ft_rn50/ww-sum/sum_wd.csv', index=False)

sum_wd_ft.to_csv('ft_rn50/ww-sum/ft_sum_wd.csv', index=False)

In [ ]:
plt.figure(figsize=(15, 5))

# Plot Accuracy

baseline_loss = test_loss
baseline_accuracy = test_acc
baseline_ece = test_ece

plt.figure(figsize=(15, 5))

# Plot Accuracy
plt.subplot(1, 3, 1)
plt.plot(weight_decay_values, [test_acc_dict_wd[i] for i in weight_decay_values], marker='o', label='Accuracy')
plt.axhline(y=baseline_accuracy, color='black', linestyle='--', label='Baseline Accuracy')
plt.title('Accuracy vs Dropout Rate')
plt.xlabel('Dropout Rate')
plt.ylabel('Accuracy (%)')
plt.legend()

# Plot Loss
plt.subplot(1, 3, 2)
plt.plot(weight_decay_values, [test_loss_dict_wd[i] for i in weight_decay_values], marker='o', label='Loss')
plt.axhline(y=baseline_loss, color='black', linestyle='--', label='Baseline Loss')
plt.title('Loss vs Dropout Rate')
plt.xlabel('Dropout Rate')
plt.ylabel('Loss')
plt.legend()

# Plot ECE
plt.subplot(1, 3, 3)
plt.plot(weight_decay_values, [test_ece_dict_wd[i] for i in weight_decay_values], marker='o', label='ECE')
plt.axhline(y=baseline_ece, color='black', linestyle='--', label='Baseline ECE')
plt.title('ECE vs Dropout Rate')
plt.xlabel('Dropout Rate')
plt.ylabel('ECE')
plt.legend()

plt.tight_layout()
plt.show()

DIV

In [11]:
auto = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

rand = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.RandAugment(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

augM = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.AugMix(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


trAu = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


class GaussianNoise(object):
    def __init__(self, mean=0.0, sigma=1.0, noise_ratio=1.0):
        self.mean = mean
        self.sigma = sigma
        self.noise_ratio = noise_ratio

    def __call__(self, img):
        # Convert PIL Image to Tensor
        img = transforms.ToTensor()(img)

        # Generate Gaussian noise
        noise = torch.randn(img.size()) * self.sigma + self.mean

        # Apply noise to the image
        noisy_img = img + noise * self.noise_ratio

        # Clip to maintain valid pixel range
        noisy_img = torch.clamp(noisy_img, 0.0, 1.0)

        # Convert back to PIL Image
        return transforms.ToPILImage()(noisy_img)

    def __repr__(self):
        return f"GaussianNoise(mean={self.mean}, sigma={self.sigma}, noise_ratio={self.noise_ratio})"

noise_01 = transforms.Compose([
    transforms.Resize((224, 224)),                   # Resizing images
    GaussianNoise(mean=0.0, sigma=0.1, noise_ratio=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


noise_03 = transforms.Compose([
    transforms.Resize((224, 224)),                   # Resizing images
    GaussianNoise(mean=0.0, sigma=0.1, noise_ratio=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


noise_05 = transforms.Compose([
    transforms.Resize((224, 224)),                     # Resizing images
    GaussianNoise(mean=0.0, sigma=0.1, noise_ratio=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

auto1 = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET),
    GaussianNoise(mean=0.0, sigma=0.1, noise_ratio=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

auto2 = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    GaussianNoise(mean=0.0, sigma=0.1, noise_ratio=0.5), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

auto3 = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.3),
    transforms.ElasticTransform(alpha=40.0, sigma=6.0, interpolation=transforms.InterpolationMode.BICUBIC),
    GaussianNoise(mean=0.0, sigma=0.1, noise_ratio=0.5), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
dataset_div = {'Auto': auto,
               'Rand': rand,
               'AugM': augM,
               'TrAu': trAu,
               'Noize10': noise_01,
               'Noize30': noise_03,
               'Noize50': noise_05,
               'Auto1': auto1,
               'Auto2': auto2,
               'Auto3': auto3
               }

# Dictionaries to store models, watchers, and summaries
model_div = {}
watcher_div = {}
details_div = {}
summary_div = {}
ft_model_div = {}
watcher_ft_div = {}
details_ft_div = {}
summary_ft_div = {}

# Dictionaries to store evaluation metrics
test_loss_dict_div = {}
test_acc_dict_div = {}
test_ece_dict_div = {}

for k, v in dataset_div.items():

    # Create custom datasets
    train_dataset_div = CustomDataset(images=train_images, labels=train_labels, classes=classes, transform=v)

    # Create data loaders
    train_dataloader_div = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Define models and watchers
    model_div[k] = ClipFinetuner(num_classes=num_classes)
    watcher_div[k] = ww.WeightWatcher(model=model_div[k])

    # Perform analysis and get summary
    details_div[k] = watcher_div[k].analyze(mp_fit=True)
    summary_div[k] = watcher_div[k].get_summary(details_div[k])

    # Save model state
    torch.save(model_div[k].state_dict(), f'ft_rn50/model/model_div{k}.pth')

    # Save analysis results to CSV
    details_div[k].to_csv(f'ft_rn50/ww-df/div{k}.csv', index=False)

    # Train the model
    ft_model_div[k] = train(model_div[k], train_dataloader_div, val_dataloader, num_epochs=epochs)

    # Evaluate the model
    test_loss, test_acc, test_ece = evaluate(ft_model_div[k], test_dataloader)
    print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
    print(f'Test ECE: {test_ece:.3f}')

    # Store metrics
    test_loss_dict_div[k] = test_loss
    test_acc_dict_div[k] = test_acc
    test_ece_dict_div[k] = test_ece

    # Analyze fine-tuned model
    watcher_ft_div[k] = ww.WeightWatcher(model=ft_model_div[k])
    details_ft_div[k] = watcher_ft_div[k].analyze(mp_fit=True)
    summary_ft_div[k] = watcher_ft_div[k].get_summary(details_ft_div[k])

    # Save fine-tuned model state
    torch.save(ft_model_div[k].state_dict(), f'ft_rn50/model/ft_model_div{k}.pth')

    # Save fine-tuned analysis results to CSV
    details_ft_div[k].to_csv(f'ft_rn50/ww-df/ft_div{k}.csv', index=False)


acc.update(test_acc_dict_div)
loss.update(test_loss_dict_div)
ece.update(test_ece_dict_div)

sum_div = pd.DataFrame([summary_div])
sum_div_ft = pd.DataFrame([summary_ft_div])

sum_div.to_csv('ft_rn50/ww-sum/sum_div.csv', index=False)

sum_div_ft.to_csv('ft_rn50/ww-sum/ft_sum_div.csv', index=False)

In [ ]:
keys = list(dataset_div.keys())

plt.figure(figsize=(15, 5))

# Plot Accuracy
plt.subplot(1, 3, 1)
plt.plot(keys, [test_acc_dict_div[k] for k in keys], marker='o', label='Accuracy')
plt.axhline(y=baseline_accuracy, color='black', linestyle='--', label='Baseline Accuracy')
plt.title('Accuracy vs Dataset Type')
plt.xlabel('Dataset Type')
plt.ylabel('Accuracy (%)')
plt.xticks(rotation=45)  # Rotate x labels for better readability
plt.legend()

# Plot Loss
plt.subplot(1, 3, 2)
plt.plot(keys, [test_loss_dict_div[k] for k in keys], marker='o', label='Loss')
plt.axhline(y=baseline_loss, color='black', linestyle='--', label='Baseline Loss')
plt.title('Loss vs Dataset Type')
plt.xlabel('Dataset Type')
plt.ylabel('Loss')
plt.xticks(rotation=45)  # Rotate x labels for better readability
plt.legend()

# Plot ECE
plt.subplot(1, 3, 3)
plt.plot(keys, [test_ece_dict_div[k] for k in keys], marker='o', label='ECE')
plt.axhline(y=baseline_ece, color='black', linestyle='--', label='Baseline ECE')
plt.title('ECE vs Dataset Type')
plt.xlabel('Dataset Type')
plt.ylabel('ECE')
plt.xticks(rotation=45)  # Rotate x labels for better readability
plt.legend()

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
acc_to = pd.DataFrame([acc])
ece_to =  pd.DataFrame([ece])
loss_to = pd.DataFrame([loss])

acc_to.to_csv('ft_rn50/acc.csv', index=False)
print(f"DataFrame saved to {'acc.csv'}")

ece_to.to_csv('ft_rn50/ece.csv', index=False)
print(f"DataFrame saved to {'ece.csv'}")

loss_to.to_csv('ft_rn50/loss.csv', index=False)
print(f"DataFrame saved to {'loss.csv'}")

VS

In [ ]:
from scripts.vs import embedding_vendi_score
from vendi_score import image_utils

pixel = {}
embedding = {}

# baseline
pxl = image_utils.pixel_vendi_score(train_images)
emb = embedding_vendi_score(train_images, device="cuda", model='RN50')
print(f"Pixel Vendi Score:{pxl:.4f}")
print(f"Embedding Vendi Score: {emb:.4f}")
pixel['bs'] = pxl
embedding['bs'] = emb

In [17]:
auto = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET)
])

rand = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.RandAugment()
])

augM = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.AugMix()
])


trAu = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.TrivialAugmentWide()
])


class GaussianNoise(object):
    def __init__(self, mean=0.0, sigma=1.0, noise_ratio=1.0):
        self.mean = mean
        self.sigma = sigma
        self.noise_ratio = noise_ratio

    def __call__(self, img):
        # Convert PIL Image to Tensor
        img = transforms.ToTensor()(img)

        # Generate Gaussian noise
        noise = torch.randn(img.size()) * self.sigma + self.mean

        # Apply noise to the image
        noisy_img = img + noise * self.noise_ratio

        # Clip to maintain valid pixel range
        noisy_img = torch.clamp(noisy_img, 0.0, 1.0)

        # Convert back to PIL Image
        return transforms.ToPILImage()(noisy_img)

    def __repr__(self):
        return f"GaussianNoise(mean={self.mean}, sigma={self.sigma}, noise_ratio={self.noise_ratio})"

noise_01 = transforms.Compose([
    transforms.Resize((224, 224)),                   # Resizing images
    GaussianNoise(mean=0.0, sigma=0.1, noise_ratio=0.1)
])


noise_03 = transforms.Compose([
    transforms.Resize((224, 224)),                   # Resizing images
    GaussianNoise(mean=0.0, sigma=0.1, noise_ratio=0.3)
])


noise_05 = transforms.Compose([
    transforms.Resize((224, 224)),                     # Resizing images
    GaussianNoise(mean=0.0, sigma=0.1, noise_ratio=0.5)
])

auto1 = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.AutoAugment(),
    GaussianNoise(mean=0.0, sigma=0.1, noise_ratio=0.5)
])

auto2 = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.AutoAugment(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    GaussianNoise(mean=0.0, sigma=0.1, noise_ratio=0.5),
])

auto3 = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.AutoAugment(),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.3),
    transforms.ElasticTransform(alpha=40.0, sigma=6.0, interpolation=transforms.InterpolationMode.BICUBIC),
    GaussianNoise(mean=0.0, sigma=0.1, noise_ratio=0.5)
])


def auto_transform(image):
    image = auto(image)  # Apply auto3 transform (assumes this returns a PIL image)
    return image

def rand_transform(image):
    image = rand(image)  # Apply auto3 transform (assumes this returns a PIL image)
    return image

def augM_transform(image):
    image = augM(image)  # Apply auto3 transform (assumes this returns a PIL image)
    return image

def trAu_transform(image):
    image = trAu(image)  # Apply auto3 transform (assumes this returns a PIL image)
    return image

def noise_01_transform(image):
    image = noise_01(image)  # Apply auto3 transform (assumes this returns a PIL image)
    return image

def noise_03_transform(image):
    image = noise_03(image)  # Apply auto3 transform (assumes this returns a PIL image)
    return image

def noise_05_transform(image):
    image = noise_05(image)  # Apply auto3 transform (assumes this returns a PIL image)
    return image

def auto1_transform(image):
    image = auto1(image)  # Apply auto3 transform (assumes this returns a PIL image)
    return image

def auto2_transform(image):
    image = auto2(image)  # Apply auto3 transform (assumes this returns a PIL image)
    return image

def auto3_transform(image):
    image = auto3(image)  # Apply auto3 transform (assumes this returns a PIL image)
    return image


In [ ]:
dataset_div = {'Auto': auto_transform,
               'Rand': rand_transform,
               'AugM': augM_transform,
               'TrAu': trAu_transform,
               'Noize10': noise_01_transform,
               'Noize30': noise_03_transform,
               'Noize50': noise_05_transform,
               'Auto1': auto1_transform,
               'Auto2': auto2_transform,
               'Auto3': auto3_transform
               }

for k, v in dataset_div.items():

    train_images_ = [v(image) for image in train_images]
    val_images_ = [v(image) for image in val_images]

    pxl = image_utils.pixel_vendi_score(train_images_)
    emb = embedding_vendi_score(train_images_, device="cuda", model='RN50')
    print(f"Pixel Vendi Score:{pxl:.4f}")
    print(f"Embedding Vendi Score: {emb:.4f}")

    pixel[k] = pxl
    embedding[k] = emb


print(pixel)
print(embedding)

In [19]:
pixel_to = pd.DataFrame([pixel])
embedding_to = pd.DataFrame([embedding])

In [ ]:
pixel_to.to_csv('ft_rn50/pixel.csv', index=False)
print(f"DataFrame saved to {'pixel.csv'}")

embedding_to.to_csv('ft_rn50/embedding.csv', index=False)
print(f"DataFrame saved to {'embedding.csv'}")

OOD

In [21]:
_, clipart_data = office_home('clipart')
_, product_data = office_home('product')
_, art_data = office_home('art')

In [22]:
clipart_images = clipart_data ['image']
clipart_labels = clipart_data['label']

product_images = product_data['image']
product_labels = product_data['label']

art_images = art_data['image']
art_labels = art_data['label']

# clipart_images = convert_bytes_to_images(clipart_images)
# product_images = convert_bytes_to_images(product_images)
# art_images = convert_bytes_to_images(art_images)

# Create custom datasets
clipart_dataset = CustomDataset(images=clipart_images, labels=clipart_labels, classes=classes, transform=transform)
product_dataset = CustomDataset(images=product_images, labels=product_labels, classes=classes, transform=transform)
art_dataset = CustomDataset(images=art_images, labels=art_labels, classes=classes, transform=transform)

# Create data loaders
clipart_dataloader = DataLoader(clipart_dataset, batch_size=32, shuffle=False)
product_dataloader = DataLoader(product_dataset, batch_size=32, shuffle=False)
art_dataloader = DataLoader(art_dataset, batch_size=32, shuffle=False)

In [ ]:
ood_acc_clipart = {}
ood_loss_clipart = {}
ood_ece_clipart = {}

ood_acc_product = {}
ood_loss_product = {}
ood_ece_product = {}

ood_acc_art = {}
ood_loss_art = {}
ood_ece_art = {}

test_loss, test_acc, test_ece = evaluate(ft_model, clipart_dataloader)
print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
print(f'Test ECE: {test_ece:.3f}')
ood_acc_clipart['bs'] = test_acc
ood_loss_clipart['bs'] = test_loss
ood_ece_clipart['bs'] = test_ece


test_loss, test_acc, test_ece = evaluate(ft_model, product_dataloader)
print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
print(f'Test ECE: {test_ece:.3f}')
ood_acc_product['bs'] = test_acc
ood_loss_product['bs'] = test_loss
ood_ece_product['bs'] = test_ece


test_loss, test_acc, test_ece = evaluate(ft_model, art_dataloader)
print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
print(f'Test ECE: {test_ece:.3f}')
ood_acc_art['bs'] = test_acc
ood_loss_art['bs'] = test_loss
ood_ece_art['bs'] = test_ece

In [ ]:
for k, v in model_dp.items():
    print(f"Model: {k}")
    test_loss, test_acc, test_ece = evaluate(v, clipart_dataloader)
    print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
    print(f'Test ECE: {test_ece:.3f}')
    ood_acc_clipart[k] = test_acc
    ood_loss_clipart[k] = test_loss
    ood_ece_clipart[k] = test_ece


    test_loss, test_acc, test_ece = evaluate(v, product_dataloader)
    print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
    print(f'Test ECE: {test_ece:.3f}')
    ood_acc_product[k] = test_acc
    ood_loss_product[k] = test_loss
    ood_ece_product[k] = test_ece


    test_loss, test_acc, test_ece = evaluate(v, art_dataloader)
    print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
    print(f'Test ECE: {test_ece:.3f}')
    ood_acc_art[k] = test_acc
    ood_loss_art[k] = test_loss
    ood_ece_art[k] = test_ece

In [ ]:
for k, v in model_wd.items():
    print(f"Model: {k}")
    test_loss, test_acc, test_ece = evaluate(v, clipart_dataloader)
    print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
    print(f'Test ECE: {test_ece:.3f}')
    ood_acc_clipart[k] = test_acc
    ood_loss_clipart[k] = test_loss
    ood_ece_clipart[k] = test_ece


    test_loss, test_acc, test_ece = evaluate(v, product_dataloader)
    print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
    print(f'Test ECE: {test_ece:.3f}')
    ood_acc_product[k] = test_acc
    ood_loss_product[k] = test_loss
    ood_ece_product[k] = test_ece


    test_loss, test_acc, test_ece = evaluate(v, art_dataloader)
    print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
    print(f'Test ECE: {test_ece:.3f}')
    ood_acc_art[k] = test_acc
    ood_loss_art[k] = test_loss
    ood_ece_art[k] = test_ece

In [ ]:
for k, v in model_div.items():
    print(f"Model: {k}")
    test_loss, test_acc, test_ece = evaluate(v, clipart_dataloader)
    print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
    print(f'Test ECE: {test_ece:.3f}')
    ood_acc_clipart[k] = test_acc
    ood_loss_clipart[k] = test_loss
    ood_ece_clipart[k] = test_ece


    test_loss, test_acc, test_ece = evaluate(v, product_dataloader)
    print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
    print(f'Test ECE: {test_ece:.3f}')
    ood_acc_product[k] = test_acc
    ood_loss_product[k] = test_loss
    ood_ece_product[k] = test_ece


    test_loss, test_acc, test_ece = evaluate(v, art_dataloader)
    print(f'Test Loss: {test_loss:.3f}, Test Accuracy: {test_acc:.3f}%')
    print(f'Test ECE: {test_ece:.3f}')
    ood_acc_art[k] = test_acc
    ood_loss_art[k] = test_loss
    ood_ece_art[k] = test_ece

In [28]:
ood_acc_clipart_to = pd.DataFrame([ood_acc_clipart])

ood_loss_clipart_to = pd.DataFrame([ood_loss_clipart])

ood_ece_clipart_to = pd.DataFrame([ood_ece_clipart])

ood_acc_product_to = pd.DataFrame([ood_acc_product])

ood_loss_product_to = pd.DataFrame([ood_loss_product])

ood_ece_product_to = pd.DataFrame([ood_ece_product])

ood_acc_art_to = pd.DataFrame([ood_acc_art])

ood_loss_art_to = pd.DataFrame([ood_loss_art])

ood_ece_art_to = pd.DataFrame([ood_ece_art])

In [30]:
ood_acc_clipart_to.to_csv('ft_rn50/ood_acc_clipart.csv', index=False)

ood_loss_clipart_to.to_csv('ft_rn50/ood_loss_clipart.csv', index=False)

ood_ece_clipart_to.to_csv('ft_rn50/ood_ece_clipart.csv', index=False)

ood_acc_product_to.to_csv('ft_rn50/ood_acc_product.csv', index=False)

ood_loss_product_to.to_csv('ft_rn50/ood_loss_product.csv', index=False)

ood_ece_product_to.to_csv('ft_rn50/ood_ece_product.csv', index=False)

ood_acc_art_to.to_csv('ft_rn50/ood_acc_art.csv', index=False)

ood_loss_art_to.to_csv('ft_rn50/ood_loss_art.csv', index=False)

ood_ece_art_to.to_csv('ft_rn50/ood_ece_art.csv', index=False)
